In [1]:
from zhipuai import ZhipuAI
import os
from langchain_community.embeddings import ZhipuAIEmbeddings
from dotenv import load_dotenv
load_dotenv("../.env")

embd_zhipu = []

def get_embedding(text, platform, model):
    text = text.replace("\n", " ")
    return platform.embeddings.create(input=text,model=model).data[0].embedding


def get_zhipu_embeddings_list(text_list):
    zhipuai = ZhipuAI(api_key=os.environ.get("ZHIPU_API_KEY"))
    for text in text_list:
        embd = get_embedding(text, zhipuai, "embedding-2")
        embd_zhipu.append(embd)
    
    
# 用于知识库
def get_zhipu_embeddings_docs(texts):
    embeddings = ZhipuAIEmbeddings(
        api_key=os.environ.get("ZHIPUAI_API_KEY")
    )
    return embeddings.embed_documents(texts)


# 用于查询
def get_zhipu_embeddings_queries(texts):
    embeddings = ZhipuAIEmbeddings(
        api_key=os.environ.get("ZHIPU_API_KEY")
    )
    return embeddings.embed_documents(texts)

print(len(get_zhipu_embeddings_docs(["hello world", "hello"])[0]))
print(len(get_zhipu_embeddings_queries(["hello world"])[0]))

1024
1024


In [2]:
# 连接zilliz
from pymilvus import utility, FieldSchema, DataType, CollectionSchema, Collection, connections

connections.connect(
  alias='default', 
  uri=os.environ.get("CLUSTER_ENDPOINT"),
  token=os.environ.get("TOKEN"), 
)

In [3]:
# 新建一个collection，相当于sql里面的表
def get_schema():
    field1 = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)
    field2 = FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512)
    # field3 = FieldSchema(name="url", dtype=DataType.VARCHAR, max_length=512)
    field4 = FieldSchema(name="text_vector", dtype=DataType.FLOAT_VECTOR, dim=1024)  # 智谱embedding后是1024
    schema = CollectionSchema(fields=[field1, field2, field4])
    return schema

def recreate_collection(collection_name):
    utility.drop_collection(collection_name=collection_name)
    schema = get_schema()
    collection = Collection(name=collection_name, schema=schema)
    index_params = {
        "index_type": "AUTOINDEX",
        "metric_type": "L2",
        "params": {}
    }
    collection.create_index(
        field_name="text_vector",
        index_params=index_params,
        index_name='vector_idx'
    )
    collection.load()
    return collection

def get_collection(collection_name):
    return Collection(name=collection_name)

In [4]:
def insert_data(collection, df):
    vectors = df['embedding'].tolist()
    data = [
        {"text": f"{text}", "text_vector": vector} for text, vector in zip(df['text'], vectors)
    ]
    collection.insert(data)

In [5]:
import pandas as pd
import os
collection = recreate_collection("big_create_demo")
for file in os.listdir('csv'):
    df = pd.read_csv(f'csv/{file}')
    df['embedding'] = get_zhipu_embeddings_docs(df['text'].tolist())
    insert_data(collection, df)
    collection.flush()
    print(f"inserted {file}, total entities: {collection.num_entities}")

inserted food.csv, total entities: 11
inserted scenery.csv, total entities: 21
inserted study.csv, total entities: 31


In [6]:
def search(collection, query_embedding, top_k=5):
    search_params = {
        "metric_type": "L2",
        "params": {"level": 2}
    }
    results = collection.search(
        query_embedding,
        anns_field="text_vector",
        param=search_params,
        limit=top_k,
        output_fields=["text"]
    )
    return results

# query_embedding = get_openai_embedding(["单项奖学金"])[0]
collection = get_collection("big_create_demo")
query_embedding = get_zhipu_embeddings_queries(["想吃北餐二楼的鸡公煲"])
results = search(collection, query_embedding)

for i in results[0]:
    print(f"{i.entity.get('text')}\n")
    
print('---------------------')

query_embedding = get_zhipu_embeddings_queries(["想吃北餐二楼的麻辣鸡公煲"])
results = search(collection, query_embedding)

for i in results[0]:
    print(f"{i.entity.get('text')}\n")

那家老字号的烤鸭店，烤鸭皮脆肉嫩，搭配上甜面酱和葱丝，风味独特。

那道经典的宫保鸡丁，酸甜辣三味交织，鸡肉嫩滑，花生香脆，是下饭的好菜。

那道招牌红烧肉，色泽红亮，肥而不腻，入口即化，是传统美食的代表。

这盘色香味俱全的麻辣香锅，各种食材在麻辣的汤底中翻滚，让人欲罢不能。

这盘清蒸鲈鱼，鱼肉鲜嫩，汤汁清澈，保留了海鲜的原汁原味。

---------------------
那家老字号的烤鸭店，烤鸭皮脆肉嫩，搭配上甜面酱和葱丝，风味独特。

这盘色香味俱全的麻辣香锅，各种食材在麻辣的汤底中翻滚，让人欲罢不能。

那道经典的宫保鸡丁，酸甜辣三味交织，鸡肉嫩滑，花生香脆，是下饭的好菜。

那道招牌红烧肉，色泽红亮，肥而不腻，入口即化，是传统美食的代表。

这碗热气腾腾的牛肉面，汤头浓郁，牛肉酥烂，让人忍不住连汤带面一扫而光。



In [7]:
# question = "xxx"
# embedding = get_zhipu_embeddings_queries(question)
# results = search(collection, [embedding])
# context = ""
# for i in results[0]:
#     context += f"{i.entity.get('text')}\n"
# 
# template = f'''你是回答问题的助理。使用以下检索到的上下文来回答问题，在回复答案时请携带原文链接（如有）。如果你不知道答案，就说你不知道。
# 上下文: {context} 
# 问题: {question} 
# 回答:'''

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Milvus
from langchain_core.runnables import RunnablePassthrough

os.environ["OPENAI_API_KEY"] = "EMPTY"
os.environ["OPENAI_API_BASE"] = 'https://862a-101-7-169-63.ngrok-free.app/v1/'

vectorstore = Milvus(
    embedding_function = ZhipuAIEmbeddings(model="embedding-2"),
    collection_name = "big_create_demo",
    connection_args={
        "uri": os.environ.get("CLUSTER_ENDPOINT"),
        "token": os.environ.get("TOKEN"),  # API key, for serverless clusters which can be used as replacements for user and password
        "secure": True,
    },
    primary_field="id",
    text_field="text",
    vector_field="text_vector",
)
retriever = vectorstore.as_retriever(search_kwargs=dict(k=5))

template = '''你是回答问题的助理。使用以下检索到的上下文来回答问题，在回复答案时请携带原文（如有）。如果你不知道答案，就说你不知道。
上下文: {context} 
问题: {question} 
回答:'''

llm = ChatOpenAI()
prompt = PromptTemplate.from_template(template=template)

rag_chain = (
    # input question
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

res = rag_chain.invoke("明天的甜点吃什么比较好呢？")
res.content

D:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 0.3.0. An updated version of the class exists in the langchain-milvus package and should be used instead. To use it run `pip install -U langchain-milvus` and import as `from langchain_milvus import MilvusVectorStore`.
  warn_deprecated(


"明天的甜点可以考虑吃新鲜采摘的草莓搭配上自制的奶油，做成的草莓奶油蛋糕，它甜而不腻，是夏日里的一抹清凉。[Document(metadata={'id': 451225562374003371}, page_content='新鲜采摘的草莓搭配上自制的奶油，做成的草莓奶油蛋糕，甜而不腻，是夏日里的一抹清凉。')]"